In [ ]:
import transformers, torch, datasets
print("transformers", transformers.__version__)
print("torch", torch.__version__)
print("datasets", datasets.__version__)

Подгружаем готовый датасет sst2 из HuggingFace Datasets Hub 

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

dataset = load_dataset("sst2")

tokenizer - токенизатор для предобработки данных. Токенизируем набор данных - разбиваем его на компоненты понятного для модели вида

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Загрузим предобученную модель. В переменной model будет предобученная модель distilBERT. Указываем устройство обучения (Cuda, если есть видеокарта Nvidia, иначе ЦП)

In [ ]:
from torch import device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)


Задаем параметры обучения

In [ ]:
training_args = TrainingArguments(
    output_dir="~/test",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Запускаем обучение сети

In [ ]:
trainer.train()

Считаем метрики

In [ ]:
model.eval()